<a href="https://www.kaggle.com/code/minhcng/sentiment-analysis-using-roberta-lstm?scriptVersionId=137316271" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
!pip install evaluate
!pip install py_vncorenlp
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.1 MB/s eta 0:00:00a 0:00:01
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4324 sha256=0d36c331cc35b59fd3397b1d4905b8b557b45299ac28257863d97d1c7b59f3e3
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-gr6uh7dd
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-gr6uh7dd
  Resolved https://github.com/huggingface/transformers.git to commit 6112b1c6442aaf7affd2b0676a1cd4eee30c45cf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transf

In [4]:
import py_vncorenlp
py_vncorenlp.download_model(save_dir="./")

VnCoreNLP model folder . already exists! Please load VnCoreNLP from this folder!


In [ ]:
!mkdir dataset
!mkdir dataset/training
!mkdir output
!mkdir output/checkpoint
!mkdir output/logging

In [6]:
import os

CURRENT_PATH = os.getcwd()
TRAINING_DATASET_PATH = f"{CURRENT_PATH}/dataset"
OUTPUT_DIR = f"{CURRENT_PATH}/output"
RAW_DATASET_PATH = "../input/vietnamese-sentiment-analyst/data - data.csv"


# Cleaning Dataset

In [7]:
import pandas as pd
dataset = pd.read_csv(RAW_DATASET_PATH)

In [8]:
dataset.shape

(31460, 4)

In [9]:
dataset.columns

Index(['comment', 'label', 'rate', 'Unnamed: 3'], dtype='object')

In [10]:
dataset.drop(columns=["Unnamed: 3", "rate"], inplace=True, axis=1)
dataset.columns

Index(['comment', 'label'], dtype='object')

In [11]:
dataset["label"].value_counts()

POS    20093
NEG     6669
NEU     4698
Name: label, dtype: int64

In [12]:
dataset=dataset.drop_duplicates("comment")
dataset = dataset.dropna()

In [13]:
dataset["label"].value_counts()

POS    16222
NEG     6330
NEU     4251
Name: label, dtype: int64

In [14]:
dataset["comment"].sample(40)

30617    Không như vài khách trc mua chê áo rộng thùng ...
2528     Điểm trừ duy nhất mình mua huarache đỏ đế trắn...
5263     Chất lượng sản phẩm tuyệt vời Quần đep y hinh ...
30688    I Chất lượng sản phẩm tuyệt vời Thời gian giao...
915                                     Sp ok chất lg tốt.
21113    chất lượng sp tốt đẹp hơn hình thời gian giao ...
31141    4 áo có 2 loại mỏng và dày.1 áo màu hồng khác ...
22907                                     Rất đáng yêu.😁😀.
22559    Sản phẩm đạt, nhưng không hài lòng vấn đề cho ...
28162                                    Áo đẹp y hình 😍😍.
2545                         Hơi to so với cân nặng chuẩn.
20605          Hàng thật không giống với hình ảnh đưa lên.
29141           Rộng quá shop cho em đổi lại aiize s vs ạ.
11449    Đóng gói sản phẩm rất đẹp và chắc chắn Shop ph...
24710                            Nghe được chuẩn lossless.
11084                   chất ok.may dep.sẽ ủng hộ shop nh.
19128    Chất lượng sản phẩm tuyệt vời , shop rất dễ th.

In [15]:
import regex as re
import string
import json

emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)

def clean_text(text):
    text = text.lower()
    text = re.sub(emoji_pattern, " ", text)
    text = re.sub(r'([a-z]+?)\1+',r'\1', text)
    text = re.sub(r"(\w)\s*([" + string.punctuation + "])\s*(\w)", r"\1 \2 \3", text)
    text = re.sub(r"(\w)\s*([" + string.punctuation + "])", r"\1 \2", text)
    # text = re.sub(r"(\d)([^\d.])", r"\1 \2", text)
    # text = re.sub(r"([^\d.])(\d)", r"\1 \2", text)
    text = re.sub(f"([{string.punctuation}])([{string.punctuation}])+",r"\1", text)
    text = text.strip()
#     text = " ".join([common_mapping.get(word, word) for word in text.split()])
    while text.endswith(tuple(string.punctuation+string.whitespace)):
        text = text[:-1]
    while text.startswith(tuple(string.punctuation+string.whitespace)):
        text = text[1:]
    text = re.sub(r"\s+", " ", text)
    return text

def map_label(label):
    label_map = {
        "POS": 0,
        "NEG": 1,
        "NEU": 2
    }
    return label_map[label]

def map_label1(label):
    label_map = {
        "Enjoyment"   : 0,
        "Disgust"     : 1,
        "Other"       : 2,
        "Sadness"     :  1,
        "Anger"       :  1,
        "Fear"        :  1,
        "Surprise"   :   0,
    }
    return label_map[label]

In [16]:
dataset["comment"] = dataset["comment"].map(lambda text: clean_text(text))
dataset["label"] = dataset["label"].map(lambda label: map_label(label))

In [17]:
dataset["comment"].sample(40)

14638    đóng gói sản phẩm rất đẹp và chắc chắn chất lư...
5170                             áo màu lên đẹp , rất sáng
28986          tại sao máy xay hoa quả giao thiếu khay lọc
30881                        tuy nhiên size ko giống mô tả
21415        chất liệu ổn , mẫu mã ổn shop phục vụ rất tốt
13455                      có dịp shery sẽ ủng hộ tiếp nha
29552    chất vải mát chất lượng sản phẩm tuyệt vời đón...
617                            màu tím vỏ ngoài mo hết chu
17101    quá ngon , gắn vô ăn luôn , mong shop có thêm ...
8999                                              rất mỏng
11876    shop cũng để lại giấy nhớ với dòng nhắn gửi si...
21974                                        dễ thương xỉu
25325    hàng đẹp và tốt chỉ 150k mà rất đẹp cám ơn sho...
19488                   với giá này thì thắt lưng tạm được
2703                               sản phẩm chất lượng tốt
17084     trong ảnh mặc qua gối mà sp nhận dc mặc trên gối
5924                    combo 10c nhưng các màu đều dễ m

In [18]:
dataset=dataset.drop_duplicates("comment")
dataset = dataset.dropna()

In [19]:
dataset = dataset.drop(dataset[dataset["comment"].map(len) < 2].index)

In [20]:
dataset["label"].value_counts()

0    15248
1     6213
2     4138
Name: label, dtype: int64

In [21]:
import random
_dataset = {
    "train": [],
    "eval": [],
    "test": []
}
temp_dataset = [(data["comment"], data["label"]) for _, data in dataset.iterrows()]
random.seed(1234)
random.shuffle(temp_dataset)

In [22]:
temp_dataset[:5]

[('vì mấy đánh giá 5 * mà mình tin tưởng lầm', 1),
 ('đổi sang rêu', 2),
 ('ngon mà', 0),
 ('chất lượng sản phẩm tuyệt vời , đẹp xuất sắc', 0),
 ('áo size s rồi nhưng thật sự còn quá rộng so với mình , chất đẹp , mát , nhân viên phục vụ tốt , giao hàng chậm , chủ shop đáng yêu',
  0)]

In [23]:
split_ratio = {
    "train": 0.8,
    "eval": 0.9,
    "test": 1
}

_i = 0
_len = len(temp_dataset)
for mode in _dataset.keys():
    _dataset[mode] = temp_dataset[_i:int(_len*split_ratio[mode])+1]
    _i = int(_len*split_ratio[mode])+1

In [24]:
from collections import Counter
import json

for mode in _dataset.keys():
    print(len(_dataset[mode]))
    a = Counter([e[1] for e in _dataset[mode]])
    print(a)
    json.dump({"data":_dataset[mode]}, open(f"{TRAINING_DATASET_PATH}/training/{mode}.json", "w", encoding="utf8"), indent=4, ensure_ascii=False)

20480
Counter({0: 12203, 1: 4994, 2: 3283})
2560
Counter({0: 1492, 1: 617, 2: 451})
2559
Counter({0: 1553, 1: 602, 2: 404})


# Training

## Import packages

In [25]:
from transformers import AutoModel, AutoTokenizer, RobertaForSequenceClassification
from transformers import Trainer, SchedulerType
from transformers.training_args import TrainingArguments, OptimizerNames
from transformers.trainer_utils import IntervalStrategy
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import py_vncorenlp
import json
import random

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

## Config

In [26]:
!pip install vncorenlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 38.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=d09c94ff3633dffcc455aec72d5528fdc55d27b173a6a9c99fb4d0b37974255f
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp


In [84]:
from vncorenlp import VnCoreNLP
class PipelineConfig:
    rdrsegmenter = VnCoreNLP("./VnCoreNLP-1.2.jar", annotators="wseg")

In [85]:
PipelineConfig.rdrsegmenter.tokenize("Tôi là một kỹ sư AI")

[['Tôi', 'là', 'một', 'kỹ_sư', 'AI']]

In [83]:
class SentimentConfig:
    def __init__(
            self,
            pretrained_path: str = None,
            dataset_path: str = None,
            training_output_dir: str = None,
            training_learning_rate: float = None,
            training_weight_decay: float = None,
            training_batch_size: int = None,
            training_num_epochs: int = None,
            training_save_total_limit: int = None,
            training_gradient_accumulation_steps: int = None,
            training_eval_steps: int = None,
            training_logging_steps: int = None,
            training_save_steps: int = None,
            training_logging_dir: str = None,
            training_warm_up_ratio: float = None,
            training_device: str = None,
            training_metrics: str = None,
            segmentor_dir:str = None,
            model_input_max_length:int= None,
            model_num_labels:int=None
    ):
        self.pretrained_path = pretrained_path if pretrained_path is not None else "vinai/phobert-base-v2"
        self.dataset_path = dataset_path if dataset_path is not None else f"{TRAINING_DATASET_PATH}/training"
        self.training_output_dir = training_output_dir if training_output_dir is not None else f"{CURRENT_PATH}/checkpoint/checkpoint20_1"
        self.training_logging_dir = training_logging_dir if training_logging_dir is not None else f"{OUTPUT_DIR}/logging"
        self.training_learning_rate = training_learning_rate if training_learning_rate is not None else 1.5e-5
        self.training_weight_decay = training_weight_decay if training_weight_decay is not None else 0.01
        self.training_batch_size = training_batch_size if training_batch_size is not None else 32
        self.training_num_epochs = training_num_epochs if training_num_epochs is not None else 10
        self.training_save_total_limit = training_save_total_limit if training_save_total_limit is not None else 3
        self.training_gradient_accumulation_steps = training_gradient_accumulation_steps if training_gradient_accumulation_steps is not None else 2
        self.training_eval_steps = training_eval_steps if training_eval_steps is not None else 200
        self.training_logging_steps = training_logging_steps if training_logging_steps is not None else 200
        self.training_save_steps = training_save_steps if training_save_steps is not None else 200
        self.training_warm_up_ratio = training_warm_up_ratio if training_warm_up_ratio is not None else 0.05
        self.training_device = training_device if training_device is not None else "cuda"
        self.training_metrics = training_metrics if training_metrics is not None else "f1"
        self.model_input_max_length = model_input_max_length if model_input_max_length is not None else 256
        self.model_num_labels = model_num_labels if model_num_labels is not None else 3

## Dataset

In [34]:
import random
random.seed(2023)
class SentimentDataset(Dataset):
    def __init__(self, config, mode, tokenizer):
        super().__init__()
        self.config = config
        self.tokenizer = tokenizer
        self.data = self.load_dataset(path=f"{self.config.dataset_path}/{mode}.json")
        random.shuffle(self.data)
        print(Counter([e[1] for e in self.data]))
        print(f"Load {len(self.data)} examples for {mode} dataset")

    def load_dataset(self, path: str):
        data = json.load(open(path, "r", encoding="utf8"))["data"]
        random.shuffle(data)
        return data

    def __getitem__(self, idx):
        data_item = self.data[idx]
        labels = torch.zeros(self.config.model_num_labels)
        labels[data_item[-1]] = 1
        text = data_item[0].lower()
        segmented_text = PipelineConfig.rdrsegmenter.tokenize(text)
        tokenized_text = self.tokenizer([" ".join(segmented_text[0])], padding="max_length", truncation=True, max_length=self.config.model_input_max_length, return_tensors="pt")
        return {
            **tokenized_text,
            "labels": labels
        }

    def __len__(self):
        return len(self.data)


## Define Trainer for training model

In [41]:
import os
os.environ["WANDB_DISABLED"] = "true"

### Trainer class

In [36]:
from typing import Optional
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers.modeling_outputs import SequenceClassifierOutput


class SentimentRobertaClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, config):
        super().__init__()
        self.lstm = nn.LSTM(config.hidden_size, config.hidden_size, 1)
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        # x = features[:, 0, :]  # take <s> token (equiv. to [CLS])
        x = features.mean(dim=1)
        x = self.dropout(x)
        x, _ = self.lstm(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class SentimentModel(RobertaForSequenceClassification):
  def __init__(self, config):
    super().__init__(config)
    self.classifier = SentimentRobertaClassificationHead(config)

  def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):

      return_dict = return_dict if return_dict is not None else self.config.use_return_dict
      outputs = self.roberta(
          input_ids,
          attention_mask=attention_mask,
          token_type_ids=token_type_ids,
          position_ids=position_ids,
          head_mask=head_mask,
          inputs_embeds=inputs_embeds,
          output_attentions=output_attentions,
          output_hidden_states=output_hidden_states,
          return_dict=return_dict,
      )
      sequence_output = outputs[0]
      logits = self.classifier(sequence_output)

      loss = None
      if labels is not None:
          # move labels to correct device to enable model parallelism
          labels = labels.to(logits.device)
          if self.config.problem_type is None:
              if self.num_labels == 1:
                  self.config.problem_type = "regression"
              elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                  self.config.problem_type = "single_label_classification"
              else:
                  self.config.problem_type = "multi_label_classification"

          if self.config.problem_type == "regression":
              loss_fct = MSELoss()
              if self.num_labels == 1:
                  loss = loss_fct(logits.squeeze(), labels.squeeze())
              else:
                  loss = loss_fct(logits, labels)
          elif self.config.problem_type == "single_label_classification":
              loss_fct = CrossEntropyLoss()
              loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
          elif self.config.problem_type == "multi_label_classification":
              loss_fct = BCEWithLogitsLoss()
              loss = loss_fct(logits, labels)

      if not return_dict:
          output = (logits,) + outputs[2:]
          return ((loss,) + output) if loss is not None else output

      return SequenceClassifierOutput(
          loss=loss,
          logits=logits,
          hidden_states=outputs.hidden_states,
          attentions=outputs.attentions,
      )

In [80]:
from evaluate import load
from transformers.trainer_utils import EvalPrediction
from collections import Counter


class SentimentTrainer:
    def __init__(self, config: SentimentConfig = None):
        self.config = config if config is not None else SentimentConfig()
#         self.model = RobertaForSequenceClassification.from_pretrained(self.config.pretrained_path, num_labels=self.config.model_num_labels)
        self.model = SentimentModel.from_pretrained(self.config.pretrained_path, num_labels=self.config.model_num_labels)
        self.model.to(self.config.training_device)
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.pretrained_path)
        self.metrics = {
            "precision": load("precision"),
            "recall": load("recall"),
            "f1": load("f1"),
            "accuracy": load("accuracy")
        }
        self.train_dataset, self.eval_dataset = self.load_dataset()

        self.init_trainer()

    def load_dataset(self):
        train_dataset = SentimentDataset(config=self.config, mode="train", tokenizer=self.tokenizer)
        eval_dataset = SentimentDataset(config=self.config, mode="eval", tokenizer=self.tokenizer)
        return train_dataset, eval_dataset

    def collator(self, data):
        batch = {}
        _keys = data[0].keys()
        for key in _keys:
            batch[key] = torch.vstack([ele[key] for ele in data]).to(self.config.training_device)

        return batch

    def compute_metrics(self, eval_predictions: EvalPrediction):
        predictions, labels = eval_predictions
        predictions = torch.from_numpy(predictions).softmax(dim=-1).argmax(dim=-1)
        labels = torch.from_numpy(labels).argmax(dim=-1)

        output =  {
            "precision": self.metrics["precision"].compute(predictions=predictions, references=labels,
                                                           average="weighted")["precision"],
            "recall": self.metrics["recall"].compute(predictions=predictions, references=labels, average="weighted")["recall"],
            "f1": self.metrics["f1"].compute(predictions=predictions, references=labels, average="weighted")["f1"],
            "accuracy": self.metrics["accuracy"].compute(predictions=predictions, references=labels)["accuracy"]
        }
        return output

    def init_trainer(self):
        training_args = TrainingArguments(
            output_dir=self.config.training_output_dir,
            evaluation_strategy=IntervalStrategy.STEPS,
            save_strategy=IntervalStrategy.STEPS,
            per_device_train_batch_size=self.config.training_batch_size,
            per_device_eval_batch_size=self.config.training_batch_size,
            learning_rate=self.config.training_learning_rate,
            weight_decay=self.config.training_weight_decay,
            save_total_limit=self.config.training_save_total_limit,
            num_train_epochs=self.config.training_num_epochs,
            gradient_accumulation_steps=2,
            eval_steps=self.config.training_eval_steps,
            fp16=self.config.training_device == "cuda",
            push_to_hub=False,
            dataloader_num_workers=0,
            logging_dir=self.config.training_logging_dir,
            logging_strategy=IntervalStrategy.STEPS,
            logging_steps=self.config.training_logging_steps,
            save_steps=self.config.training_save_steps,
            logging_first_step=False,
            optim=OptimizerNames.ADAMW_TORCH,
            load_best_model_at_end=True,
            metric_for_best_model=self.config.training_metrics,
            warmup_ratio=self.config.training_warm_up_ratio,
            lr_scheduler_type=SchedulerType.COSINE_WITH_RESTARTS,
            dataloader_pin_memory=False
        )
        self.trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=self.train_dataset,
            eval_dataset=self.eval_dataset,
            tokenizer=self.tokenizer,
            data_collator=self.collator,
            compute_metrics=self.compute_metrics
        )

    def train(self):
        self.trainer.train()

    def eval(self):
        print(self.trainer.evaluate(eval_dataset=self.eval_dataset))


### Training

In [86]:
trainer = SentimentTrainer()

Some weights of SentimentModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.lstm.bias_ih_l0', 'classifier.lstm.weight_hh_l0', 'classifier.lstm.weight_ih_l0', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.lstm.bias_hh_l0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Counter({0: 12203, 1: 4994, 2: 3283})
Load 20480 examples for train dataset
Counter({0: 1492, 1: 617, 2: 451})
Load 2560 examples for eval dataset


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
200,0.579100,0.444647,0.542625,0.666406,0.597234,0.666406
400,0.371400,0.371087,0.627923,0.756250,0.685351,0.756250


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.514600,0.287513,0.879586,0.875853,0.876103,0.875853
1000,0.296200,0.197399,0.891402,0.890161,0.890606,0.890161
1500,0.250400,0.171118,0.901920,0.897865,0.898890,0.897865
2000,0.228300,0.173790,0.898659,0.886419,0.888023,0.886419
2500,0.205800,0.174456,0.902168,0.892142,0.893699,0.892142


# EXPORT model to ONNX

In [ ]:
!pip install onnx
!pip install onnxruntime

In [ ]:
import onnx
from transformers import RobertaForSequenceClassification
from onnxruntime.transformers.optimizer import optimize_model
from transformers.convert_graph_to_onnx import quantize
import onnxruntime as ort
import torch
from functools import reduce

In [ ]:
best_checkpoint_path = f"{trainer.config.training_output_dir}/checkpoint-4500"
# best_model = RobertaForSequenceClassification.from_pretrained(best_checkpoint_path)
best_model = SentimentModel.from_pretrained(best_checkpoint_path)
best_model.eval()
tokenizer = AutoTokenizer.from_pretrained(best_checkpoint_path)

## Export

### Make saved path

In [ ]:
ONNX_FOLDER = f"{trainer.config.training_output_dir}/onnx"
!mkdir "{ONNX_FOLDER}"

### Class for converter

In [ ]:
import logging
class ONNXFunc:
    def __init__(self, dummy_inputs, model, input_names, output_names, dynamic_axes):
        self.dummy_inputs = dummy_inputs
        self.model = model
        self.input_names = input_names
        self.output_names = output_names
        self.dynamic_axes = dynamic_axes
        self.logger = logging.getLogger(self.__class__.__name__)

    @staticmethod
    def compose(*functions):
        return reduce(lambda f, g: lambda x: g(f(x)), functions)

    @staticmethod
    def check_onnx_model(path):
        model = onnx.load(path)
        onnx.checker.check_model(model)

    def onnx_only(self, saved_path):
      torch.onnx.export(
          self.model,
          self.dummy_inputs,
          saved_path,
          verbose=True,
          input_names=self.input_names,
          output_names=self.output_names,
          dynamic_axes=self.dynamic_axes
      )
      return saved_path

    def apply_optimized_func(self, saved_path: str):
      use_gpu = trainer.config.training_device=="cuda" and "CUDAExecutionProvider" in ort.get_available_providers()
      optimized_model = optimize_model(saved_path, use_gpu=use_gpu, opt_level=99, verbose=True)
      new_onnx_path = saved_path.replace(".onnx", "_optimized.onnx")
      optimized_model.save_model_to_file(new_onnx_path)
      self.logger.info(f"EXPORT model to ONNX + Optimized - DONE at {new_onnx_path}")
      return new_onnx_path

    def apply_quantized_func(self, saved_path: str):
        from pathlib import Path
        quantized_model_path = quantize(Path(saved_path))
        self.logger.info(f"EXPORT model to ONNX + Quantized - DONE at {quantized_model_path}")
        return quantized_model_path

    @property
    def mapping_func(self):
        return {
            "ONNX": self.onnx_only,
            "optimized_ONNX": self.compose(self.onnx_only, self.apply_optimized_func),
            "quantized_optimized_ONNX": self.compose(
                self.onnx_only, self.apply_optimized_func, self.apply_quantized_func)
        }

    def convert(self, option, saved_path):
        converter = self.mapping_func[option]
        converter(saved_path)

### Convert

In [ ]:
import torch
text = ["Tôi là một kĩ sư AI","Tôi là một kĩ sư AI"]
dummy_inputs = tokenizer(text, return_tensors="pt", padding=True)
input_keys = ["input_ids","attention_mask"]
dummy_inputs = tuple(dummy_inputs[i] for i in input_keys)

In [ ]:
onnx_conn = ONNXFunc(
    dummy_inputs=dummy_inputs, model=best_model, input_names=["input_ids","attention_mask"], output_names=["logits"], dynamic_axes={
        "input_ids":{
            0: "batch_size",
            1: "dim"
        },
        "attention_mask":{
            0: "batch_size",
            1: "dim"
        },
        "logits": {
            0: "batch_size"
        }
    }
)

In [ ]:
onnx_conn.convert(option="quantized_optimized_ONNX", saved_path=f"{ONNX_FOLDER}/model.onnx")

## Test and compare model

In [ ]:
examples = ["hôm nay tôi vui lắm",
            "hôm nay trời thật đẹp"]
segmented_text = [" ".join(PipelineConfig.rdrsegmenter.word_segment(example)) for example in examples ]
_inputs = tokenizer(examples, return_tensors="pt", padding=True)


### Load dataset for testing

In [ ]:
import json
test_dataset = json.load(open(f"{TRAINING_DATASET_PATH}/training/test.json", "r", encoding="utf8"))
eval_dataset = json.load(open(f"{TRAINING_DATASET_PATH}/training/eval.json", "r", encoding="utf8"))
train_dataset = json.load(open(f"{TRAINING_DATASET_PATH}/training/train.json", "r", encoding="utf8"))
len(test_dataset["data"])

2543

### Init metrics

In [ ]:
from evaluate import load
def custom_metrics(predictions, labels):
    precision = load("precision")
    recall = load("recall")
    f1 = load("f1")
    accuracy = load("accuracy")

    _precision = precision.compute(predictions=predictions, references=labels, average="weighted")
    _recall = recall.compute(predictions=predictions, references=labels, average="weighted")
    _f1 = f1.compute(predictions=predictions, references=labels, average="weighted")
    _accuracy = accuracy.compute(predictions=predictions, references=labels)
    return {
        "precision":_precision,
        "recall":_recall,
        "f1":_f1,
        "accuracy":_accuracy
    }

### Init ONNX sessions

In [ ]:
options = ort.SessionOptions()
# options.intra_op_num_threads = 1
options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
providers = ["CPUExecutionProvider"]
onnx_path=f"{ONNX_FOLDER}/model.onnx"

In [ ]:
import onnxruntime as ort
ort_session = ort.InferenceSession(onnx_path, providers=providers, sess_options=options)
quantized_model = ort.InferenceSession(f"{ONNX_FOLDER}/model_optimized-quantized.onnx", providers=providers, sess_options=options)

### Test time of sessions with one example

In [ ]:
import time

In [ ]:
t1 = time.time()
best_model(**_inputs)
print("Original model time\t\t", time.time() - t1)

ort_input = {key: _inputs[key].detach().numpy() for key in _inputs.keys() if key!="token_type_ids"}

t2 = time.time()
ort_session.run(["logits"], ort_input)
print("ONNX model time\t\t", time.time() - t2)


t4 = time.time()
quantized_model.run(["logits"], ort_input)
print("optimized + quantized ONNX model time\t\t", time.time() - t4)

Original model time		 0.36330533027648926

ONNX model time		 0.18816781044006348

optimized + quantized ONNX model time		 0.1711902618408203


### Inference accuracy and time of sessions vs base model

In [ ]:
from tqdm import tqdm
import time
import scipy
import random

In [ ]:
modes = {
    "quantized_opt_onnx":quantized_model,

    "onnx_only": ort_session,
    # "opt_onnx": opt_session,
    "base": best_model,

}

In [ ]:
import json
_len = len(eval_dataset["data"])
subset = random.choices(eval_dataset["data"], k=_len)

for mode, sess in modes.items():
    print(f"PROCESSING {mode}")
    t = time.time()
    acc = 0
    labelsss = []
    predsss = []
    for example, label in tqdm(subset):
        labelsss.append(label)
        segmented_text = [" ".join(PipelineConfig.rdrsegmenter.word_segment(example))]
        _inputs = tokenizer(segmented_text, return_tensors="pt", padding=True)
        if "onnx" in mode:
            ort_input = {key: _inputs[key].detach().numpy() for key in _inputs.keys() if key!="token_type_ids"}
            output = sess.run(["logits"], ort_input)[0]
            output = scipy.special.softmax(output, axis=-1)
            pred = output.argmax(-1).tolist()[0]
        else:
            output = sess(**_inputs)
            output = output.logits.argmax(dim=-1)
            pred = output.item()
        predsss.append(pred)
        if label == pred:
            acc += 1
    end_time=time.time()
    print(f"{mode} DONE with avg time={(end_time-t)/_len}")
    print(json.dumps(custom_metrics(predsss, labelsss), indent=4, ensure_ascii=False))

In [ ]:
import json
_len = len(eval_dataset["data"])
subset = random.choices(eval_dataset["data"], k=_len)

for mode, sess in modes.items():
    print(f"PROCESSING {mode}")
    t = time.time()
    acc = 0
    labelsss = []
    predsss = []
    for example, label in tqdm(subset):
        labelsss.append(label)
        segmented_text = [" ".join(PipelineConfig.rdrsegmenter.word_segment(example))]
        _inputs = tokenizer(segmented_text, return_tensors="pt", padding=True)
        if "onnx" in mode:
            ort_input = {key: _inputs[key].detach().numpy() for key in _inputs.keys() if key!="token_type_ids"}
            output = sess.run(["logits"], ort_input)[0]
            output = scipy.special.softmax(output, axis=-1)
            pred = output.argmax(-1).tolist()[0]
        else:
            output = sess(**_inputs)
            output = output.logits.argmax(dim=-1)
            pred = output.item()
        predsss.append(pred)
        if label == pred:
            acc += 1
    end_time=time.time()
    print(f"{mode} DONE with avg time={(end_time-t)/_len}")
    print(json.dumps(custom_metrics(predsss, labelsss), indent=4, ensure_ascii=False))

PROCESSING quantized_opt_onnx


100%|██████████| 2543/2543 [01:24<00:00, 30.26it/s]


quantized_opt_onnx DONE with avg time=0.03305233935882292

{

    "precision": {

        "precision": 0.8612241883088714

    },

    "recall": {

        "recall": 0.8607943373967755

    },

    "f1": {

        "f1": 0.8606019117950287

    },

    "accuracy": {

        "accuracy": 0.8607943373967755

    }

}

PROCESSING onnx_only


100%|██████████| 2543/2543 [02:05<00:00, 20.19it/s]


onnx_only DONE with avg time=0.04953403075379834

{

    "precision": {

        "precision": 0.8552968188951409

    },

    "recall": {

        "recall": 0.8560755013763272

    },

    "f1": {

        "f1": 0.8554314448111257

    },

    "accuracy": {

        "accuracy": 0.8560755013763272

    }

}

PROCESSING base


  9%|▊         | 219/2543 [00:38<06:45,  5.74it/s]


KeyboardInterrupt: ignored

- Model without ONNX:
    - accuracy = 0.857
    - f1 = 0.856
    - avg time = 158.717 ms/example
- ONNX only:
    - accuracy = 0.857
    - f1 = 0.856
    - time = 58.079 ms/example
- ONNX and quantize:
    - accuracy = 0.858
    - f1 = 0.856
    - avg time = 29.912 ms/example


#### Test quantized model

In [ ]:
def reverse_label(label):
    label_map = {
        0: "POS",
        1: "NEG",
        2: "NEU"
    }
    return label_map[label]

In [ ]:
_len = 1000
subset = random.choices(eval_dataset["data"], k=_len)
acc=0
for example, label in subset:
    segmented_text = [" ".join(PipelineConfig.rdrsegmenter.word_segment(example))]
    _inputs = tokenizer(segmented_text, return_tensors="pt", padding=True)

    ort_input = {key: _inputs[key].detach().numpy() for key in _inputs.keys() if key!="token_type_ids"}
    output = quantized_model.run(["logits"], ort_input)[0]
    output = scipy.special.softmax(output, axis=-1)
    pred = output.argmax(-1).tolist()[0]

    if label == pred:
        acc += 1
    else:
        print("==============")
        print(example)
        print(reverse_label(label), "!=", reverse_label(pred))


màu thì đẹp chất thun đồng ý nhung hơi nhỏ và ngắn

NEU != POS


dáng áo không đẹp bằng áo sọc lần trước mình mua

NEU != NEG


màu hơi vàng chứ không phải trắng

NEU != NEG


một thùng hàng to đùng luôn

NEU != POS


m mặc s mà mua s mặc rộng quá

NEG != NEU


riêng găng tay mỏng và dễ rách

NEG != NEU


chất áo đẹp , đóng gói đẹp nhưng k hài lòng vì vạt áo tay áo bị quăn vào còn lại rất hài lòng

NEU != POS


còn 2 áo croptop và polo thì giống hình đồng ý

NEG != POS


đóng gói sản phẩm chắc chắn thời gian giao hàng nhanh chất lượng sản phẩm tạm được đáng đồng tiền

NEU != POS


shop chú ý kiểm tra kỹ hơn

NEU != NEG


mua hàng lần thứ 2 rồi nè

NEU != POS


hang đẹp

NEU != POS


áo bị phai màu nhưng so với giá tiền thì đồng ý

POS != NEU


shop nên cho chọn size , vì đặt mua xong thì shop đưa size nhỏ nên không vừa

NEG != NEU


thời gian giao hàng chậm thời gian giao hàng chậm

NEU != NEG


áo may tuột đường chỉ từa lưa , dơ nữa chứ

NEU != NEG


mua 4 áo size người lớn mà mang đ

In [ ]:
_len = 2000
subset = random.choices(train_dataset["data"], k=_len)
acc=0
for example, label in subset:
    segmented_text = [" ".join(PipelineConfig.rdrsegmenter.word_segment(example))]
    _inputs = tokenizer(segmented_text, return_tensors="pt", padding=True)

    ort_input = {key: _inputs[key].detach().numpy() for key in _inputs.keys() if key!="token_type_ids"}
    output = quantized_model.run(["logits"], ort_input)[0]
    output = scipy.special.softmax(output, axis=-1)
    pred = output.argmax(-1).tolist()[0]

    if label == pred:
        acc += 1
    else:
        print("==============")
        print(example)
        print(reverse_label(label), "!=", reverse_label(pred))


lịch sự

NEU != POS


ốp đẹp lắm . mà có điều không bo khít vào thân máy cho lắm

NEU != POS


chất lượng sản phẩm tốt . nhưng giao thiếu dây cột chỗ tay áo

NEU != POS


quần và áo gói trong túi zip riêng khá chất lượng

NEG != POS


ao cung size voi ao gia dinh nhung thuc te nho hon 2cm nen khong mac vua

NEG != NEU


kem bên trong là màu trắng khác với kem ngày là màu vàng nhẹ . thời gian giao hàng chậm

NEU != NEG


đóng gói sản phẩm rất đẹp và chắc chắn . ưng ý ạ

NEU != POS


49k - hàng như hình , loại dry 48h mùi k đc ưng lắm

NEG != NEU


vải đẹp quá

NEU != POS


chất lượng sản phẩm tốt thời gian giao hàng chậm

NEU != POS


nhưng giá tiền này thì được

POS != NEU


váy với áo đều đồng ý

NEU != POS


mua nhiều mà shop không có km gì hết trơn ạ . buồn woa

NEU != NEG


size áo quá nhỏ

NEU != NEG


size l mà như xl

NEU != NEG


mặc êm nhưng mặc 34 lấy size s hơi chật

NEG != NEU


đã nhận được hàng rồi , đồ thì đẹp lắm nhưng tiền phí cao quá , tiền phí cao hơn tiền đồ nữa , 